In [1]:
import pandas as pd
import conllu as cl
from os.path import join

In [3]:
def load(language):
    filePath = ".\Datasets\\" + language + "\\"
    
    with open(join(filePath, "train.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    train_sentences = cl.parse(data)

    with open(join(filePath, "test.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    test_sentences = cl.parse(data)

    with open(join(filePath, "val.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    val_sentences = cl.parse(data)

    return train_sentences, test_sentences, val_sentences

In [39]:
def HMMFit(train_sentences):
    countTag = {}
    transitionCount = {}
    wordTagCount = {}

    #description
    for sentence in train_sentences:
        for word in sentence:
            if word['lemma'] in countTag:
                countTag[word['lemma']] += 1
            else:
                countTag[word['lemma']] = 1
                transitionCount[word['lemma']] = {}
            
            if word['form'] not in wordTagCount:
                wordTagCount[word['form']] = {}
            if word['lemma'] in wordTagCount[word['form']]:
                wordTagCount[word['form']][word['lemma']] += 1
            else:
                wordTagCount[word['form']][word['lemma']] = 1

    #description
    for tag in transitionCount.keys():
        for sentence in train_sentences:
            previousTag = None
            for word in sentence:
                if previousTag == None:
                    previousTag = word['lemma']
                else:
                    if word['lemma'] == tag:
                        if previousTag in transitionCount[tag]:
                            transitionCount[tag][previousTag] +=1
                        else:
                            transitionCount[tag][previousTag] = 1
                    previousTag = word['lemma']
                        
    return countTag, transitionCount, wordTagCount

In [4]:
it_train_sentences, it_test_sentences, it_val_sentences = load("it")
#en_train_sentences, en_test_sentences, en_val_sentences = load("en")
#es_train_sentences, es_test_sentences, es_val_sentences = load("es")

In [40]:
HMMFit(it_train_sentences)

{'Nella': {'O': 1186, 'B-MISC': 3},
 'stagione': {'O': 2533},
 '2003': {'O': 315, 'I-MISC': 10, 'B-MISC': 1},
 '-': {'O': 6554, 'I-MISC': 574, 'I-LOC': 7, 'I-ORG': 1},
 '2004': {'O': 381, 'I-MISC': 14, 'B-MISC': 6},
 'vince': {'O': 349},
 "un'": {'O': 2391, 'I-MISC': 2},
 'altra': {'O': 538, 'I-MISC': 4},
 'Supercoppa': {'B-MISC': 56},
 'italiana': {'I-MISC': 32, 'O': 508, 'I-LOC': 1, 'I-ORG': 2},
 'a': {'O': 30067, 'I-MISC': 173, 'I-ORG': 9, 'I-LOC': 22},
 'New': {'B-LOC': 756,
  'B-ORG': 256,
  'I-LOC': 6,
  'B-MISC': 22,
  'I-MISC': 38,
  'I-PER': 1,
  'I-ORG': 1},
 'York': {'I-LOC': 595,
  'I-ORG': 155,
  'B-LOC': 5,
  'I-MISC': 19,
  'I-PER': 6,
  'B-ORG': 2},
 'contro': {'O': 3023, 'I-MISC': 9},
 'il': {'O': 38460, 'I-MISC': 95, 'I-PER': 106, 'B-MISC': 4, 'B-PER': 1},
 'Milan': {'B-ORG': 71, 'B-PER': 7, 'I-ORG': 1},
 ',': {'O': 122540, 'I-MISC': 117, 'I-ORG': 19, 'I-PER': 2},
 'ai': {'O': 3068, 'I-MISC': 6},
 'rigori': {'O': 30},
 'realizzando': {'O': 85},
 'nel': {'O': 12719, 'I